In [14]:
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.clustering import KMeansModel
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark.pandas as ps
ps.set_option('compute.default_index_type', 'distributed')
item_path ='data/item_predictions/item_preds.csv' 
collab_path ='data/collab_predictions/collab_preds.csv'

## Verify models can be run

In [15]:
ALS_model = ALSModel.load("models/als_explicit_collab")

In [16]:
kmeans = KMeansModel.load('models/kmeans')

In [5]:
collab_preds = ps.read_csv(collab_path)

In [6]:
collab_preds.head()

,userId,streamId,interactionTime,streamerId,prediction
0,1,33842865744,2.03125,3047137,1.523179
1,1,33846768288,3.06250,3038334,1.135493
2,1,33886469056,1.00000,3047137,1.523179
3,1,33887624992,2.03125,3046060,1.402077
4,1,33890145056,3.06250,3050226,1.872689


In [7]:
item_preds = ps.read_csv(item_path)

In [8]:
item_preds.head()

,streamerId,avgInteractionTime,interactionCounts,prediction,avgInteractionTimeScaled,interactionCountsScaled
0,2987163,3.338468,71,0,1.290675,0.282402
1,3040638,3.506663,469,0,1.355700,1.865443
2,3050662,5.468750,63,0,2.114256,0.250582
3,3027657,2.075948,323,0,0.802576,1.284730
4,17971,20.078125,6,0,7.762341,0.023865


In [9]:
collab_preds =collab_preds.rename({'prediction': 'collab_preds'}, axis=1)
collab_preds.head()

,userId,streamId,interactionTime,streamerId,collab_preds
0,1,33842865744,2.03125,3047137,1.523179
1,1,33846768288,3.06250,3038334,1.135493
2,1,33886469056,1.00000,3047137,1.523179
3,1,33887624992,2.03125,3046060,1.402077
4,1,33890145056,3.06250,3050226,1.872689


In [10]:
item_preds =item_preds.rename({'prediction': 'item_preds'}, axis=1)
item_preds.head()

,streamerId,avgInteractionTime,interactionCounts,item_preds,avgInteractionTimeScaled,interactionCountsScaled
0,2987163,3.338468,71,0,1.290675,0.282402
1,3040638,3.506663,469,0,1.355700,1.865443
2,3050662,5.468750,63,0,2.114256,0.250582
3,3027657,2.075948,323,0,0.802576,1.284730
4,17971,20.078125,6,0,7.762341,0.023865


In [11]:
collab_preds.merge(item_preds, on='streamerId', how='inner', suffixes=('_1', '_2'))

,userId,streamId,interactionTime,streamerId,collab_preds,avgInteractionTime,interactionCounts,item_preds,avgInteractionTimeScaled,interactionCountsScaled
0,1,33842865744,2.03125,3047137,1.523179,2.605432,5706,1,1.007278,22.695565
1,1,33846768288,3.06250,3038334,1.135493,2.031250,225,0,0.785295,0.894936
2,1,33886469056,1.00000,3047137,1.523179,2.605432,5706,1,1.007278,22.695565
3,1,33887624992,2.03125,3046060,1.402077,2.402970,3859,3,0.929005,15.349139
4,1,33890145056,3.06250,3050226,1.872689,3.075806,1705,4,1.189128,6.781623
5,1,33903958784,3.06250,3038141,1.122724,1.984448,639,4,0.767201,2.541617
6,1,33929318864,15.43750,3050209,1.664883,2.804828,3682,3,1.084366,14.645123
7,1,33942837056,2.03125,3046060,1.402077,2.402970,3859,3,0.929005,15.349139
8,1,33955351648,2.03125,3050209,1.664883,2.804828,3682,3,1.084366,14.645123
9,1,34060922080,1.00000,3047137,1.523179,2.605432,5706,1,1.007278,22.695565


In [12]:
item_preds.loc[item_preds['streamerId'] == 412]

,streamerId,avgInteractionTime,interactionCounts,item_preds,avgInteractionTimeScaled,interactionCountsScaled
8589967052,412,1.0,1,0,0.386607,0.003977


In [13]:
temp =collab_preds.merge(item_preds, on='streamerId', how='inner', suffixes=('_1', '_2'))
temp.to_spark().coalesce(1).write.format('csv').option('header', 'true').save('data/merged_predictions')